### Importing Modules

In [ ]:
# Commom Includes
from datetime import datetime

# ML Modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pickle

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import TargetEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

: 

In [ ]:
df = pd.read_csv("../DataSet/cars24_car-price-clean-data.csv", index_col="Unnamed: 0")

: 

In [ ]:
df.columns

: 

In [ ]:
categCols = ["make", "model", "seller_type", "fuel_type", "transmission_type", "seats"]
contCols = ["year", "km_driven", "mileage", "engine", "max_power"]

: 

In [ ]:
## splitting the Data to Train-Validation-Test

randomState = 50
## spliting the data to features and target Variables
X = df.drop(columns="selling_price")
y = df["selling_price"]

X_tr_vl, X_test, y_tr_vl, y_test = train_test_split(X, y, test_size=0.2, random_state=randomState)
X_train, X_val, y_train, y_val = train_test_split(X_tr_vl, y_tr_vl, test_size=0.25, random_state=randomState)

print("Training Data size: X: %s, y: %s" % (X_train.shape, y_train.shape))
print("Validation Data size: X: %s, y: %s" % (X_val.shape, y_val.shape))
print("Test Data size: X: %s, y: %s" % (X_test.shape, y_test.shape))

: 

In [ ]:
# Scaling Model Names
X_MinMaxModelName = "minmaxModel"
X_StdModelName = "stdModel"
X_TgtModelName = "tgtModel"

# Model pickle file Names
X_MinMax_scale_pkl = "../Artifacts/X_MinMax_scale.pkl" 
X_Std_scale_pkl = "../Artifacts/X_Std_scale.pkl" 
X_Tgt_encode_pkl = "../Artifacts/X_Tgt_encode.pkl" 
y_Std_scale_pkl = "../Artifacts/y_Std_scale.pkl" 

# Scaling Continuous Data
# TODO: Need to figure out which ones to pass to Standard Scaler and vice-versa 
minmax_features = []
standard_features = contCols
Target_features = categCols

# Create and fit model for continuous features
preprocessor = ColumnTransformer(
    transformers=[
        (X_MinMaxModelName, MinMaxScaler(), minmax_features),
        (X_StdModelName, StandardScaler(), standard_features),
        (X_TgtModelName, TargetEncoder(), Target_features)],
        remainder="passthrough")
preprocessor.fit(X=X_train, y=y_train)

# Create and fit model for Target Variable
yStdScaler = StandardScaler()
yStdScaler.fit(y_train.values.reshape(-1, 1))

with open(X_MinMaxModelName, "wb") as pfile:
    pickle.dump(preprocessor.named_transformers_[X_MinMaxModelName], pfile)

with open(X_Std_scale_pkl, "wb") as pfile:
    pickle.dump(preprocessor.named_transformers_[X_StdModelName], pfile)

with open(X_TgtModelName, "wb") as pfile:
    pickle.dump(preprocessor.named_transformers_[X_TgtModelName], pfile)

with open(y_Std_scale_pkl, "wb") as pfile:
    pickle.dump(yStdScaler, pfile)

: 

In [ ]:
## now transforming the continuous Features
pd.DataFrame(preprocessor.transform(X_train), columns = preprocessor.get_feature_names_out())


: 

: 

: 

: 